In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 1.0 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp38-cp38-macosx_10_9_x86_64.whl size=800896 sha256=8b6fb98a1aa27f12fe950f21cd64904b059def1ee745a10a20a07b3c598afba9
  Stored in directory: /Users/Olga/Library/Caches/pip/wheels/e9/cd/20/eea71eed0ec3459fd035a9a982d8639c9ea44653c34f4f01d0
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [9]:
total_sales = data_train['item_id'].value_counts()
items_weights = pd.DataFrame({'item_id': total_sales.axes[0], 'weights': data_train['item_id'].value_counts() / total_sales.sum()})

In [37]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id'].values
    weights = items_weights['weight'].values

    recs = np.random.choice(items, size=n, replace=False, p=weights)

    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [39]:
item_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights['weight'] = item_weights['sales_value']/np.sum(item_weights['sales_value'])
item_weights.head()

,item_id,sales_value,weight
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07
2,26093,1.59,2.254623e-07
3,26190,1.54,2.183723e-07
4,26355,1.98,2.807644e-07


In [40]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights=item_weights, n=5))

CPU times: user 3.43 s, sys: 32.3 ms, total: 3.46 s
Wall time: 3.47 s


In [41]:
result.head()

,user_id,actual,random_recommendation,random_recommendation_2,popular_recommendation,popular_recommendation_2,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1075070, 10121905, 13213251, 845814, 6391330]","[1043128, 927681, 1048462, 1117358, 913144]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[16766811, 9245106, 898869, 1000237, 6534178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2151060, 1102154, 862751, 12428377, 2543415]","[1072968, 834382, 1039589, 1027874, 915493]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13155046, 851819, 10149640, 6979253, 918921]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6638845, 121989, 1332913, 196391, 962321]","[845208, 1013572, 1094352, 878442, 860776]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1046865, 13003094, 844179, 1023095, 887618]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5591611, 1075379, 6979513, 9827666, 9527622]","[9245106, 948495, 1127025, 5995129, 954722]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9803011, 1093490, 1076875, 13671775, 910032]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[210740, 1536176, 844560, 6944612, 1081496]","[967760, 1034686, 1004909, 8293732, 900697]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1068291, 6534166, 5569845, 1029624, 12385227]"


In [44]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [45]:
for name_col in result.columns[1:]:
#     print(name_col)
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0004:random_recommendation
0.0069:random_recommendation_2
0.1552:popular_recommendation
0.1552:popular_recommendation_2
0.026:weighted_random_recommendation


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [16]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [17]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [19]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [20]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [21]:
data_train_top = data_train.loc[data_train['item_id'].isin(top_5000)]
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [22]:
data_train2 = data_train.loc[data_train['item_id'].isin(top_5000),]

In [25]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [26]:
%%time

items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 4.95 s, sys: 38.7 ms, total: 4.99 s
Wall time: 5 s


In [27]:
%%time

items = data_train2.item_id.unique()
result['random_recommendation_2'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 436 ms, sys: 35.4 ms, total: 471 ms
Wall time: 471 ms


In [28]:
%%time

popular_recs = popularity_recommendation(data_train, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 228 ms, sys: 78.1 ms, total: 306 ms
Wall time: 309 ms


In [29]:
%%time

popular_recs = popularity_recommendation(data_train2, n=5)
result['popular_recommendation_2'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 93.1 ms, sys: 55 ms, total: 148 ms
Wall time: 147 ms


In [42]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights=item_weights, n=5))
result.head()

CPU times: user 3.73 s, sys: 44.2 ms, total: 3.77 s
Wall time: 3.83 s


,user_id,actual,random_recommendation,random_recommendation_2,popular_recommendation,popular_recommendation_2,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1075070, 10121905, 13213251, 845814, 6391330]","[1043128, 927681, 1048462, 1117358, 913144]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8090993, 5568721, 13512717, 13877096, 844165]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2151060, 1102154, 862751, 12428377, 2543415]","[1072968, 834382, 1039589, 1027874, 915493]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[822936, 6391137, 12984573, 938596, 866211]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6638845, 121989, 1332913, 196391, 962321]","[845208, 1013572, 1094352, 878442, 860776]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1059902, 871680, 10121833, 925579, 867931]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5591611, 1075379, 6979513, 9827666, 9527622]","[9245106, 948495, 1127025, 5995129, 954722]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1037863, 1037840, 1030040, 5568378, 9527290]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[210740, 1536176, 844560, 6944612, 1081496]","[967760, 1034686, 1004909, 8293732, 900697]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9829966, 875413, 989069, 962185, 1087362]"


In [46]:
pd.DataFrame([
    result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['random_recommendation_2'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['popular_recommendation_2'], row['actual']), axis=1).mean(),
], 
    columns=['precision_at_k'], 
    index=['random_recommendation', 'random_recommendation_5к', 'popular_recommendation', 'popular_recommendation_5к']
).T

,random_recommendation,random_recommendation_5к,popular_recommendation,popular_recommendation_5к
precision_at_k,0.000392,0.006856,0.15524,0.15524


In [47]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']

In [49]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id',
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [51]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [52]:
%%time

for k in range(1, 16):
    
    model = ItemItemRecommender(K=k, num_threads=4)  # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
    
    recs = model.recommend(
        userid=userid_to_id[2],  # userid - id от 0 до N
        user_items=csr_matrix(user_item_matrix).tocsr(),  # на вход user-item matrix
        N=5,  # кол-во рекомендаций 
        filter_already_liked_items=False,
        filter_items=None,
        recalculate_user=True)
    
    result[f'itemitem_K={k}'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

...

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 10min 54s, sys: 33.1 s, total: 11min 27s
Wall time: 11min 47s


In [53]:
pd.DataFrame(
    [
        result.apply(lambda row: precision_at_k(row['itemitem_K=1'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=2'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=3'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=4'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=5'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=6'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=7'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=8'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=9'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=10'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=11'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=12'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=13'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=14'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=15'], row['actual']), axis=1).mean(),
    ], columns = ['precision_at_k'], index=[f'k={i}' for i in range(1, 16)]).sort_values(by='precision_at_k', ascending=False).T

,k=1,k=2,k=3,k=15,k=10,k=11,k=9,k=14,k=12,k=13,k=8,k=7,k=6,k=5,k=4
precision_at_k,0.219491,0.206268,0.170029,0.167091,0.166993,0.166993,0.166895,0.166895,0.166601,0.166308,0.164643,0.164153,0.15906,0.154065,0.151028
